# AoC 2024 Day 15
https://adventofcode.com/2024/day/15

In [37]:
import copy

In [43]:
with open('data/day15.txt') as f:
    data = f.read()

MAP, INSTR = data.split('\n\n')
MAP = [list(l) for l in MAP.splitlines()]
INSTR = INSTR.replace('\n', '')

In [44]:
MAP

[['#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#',
  '#'],
 ['#',
  'O',
  'O',
  'O',
  '#',
  '.',
  '.',
  'O',
  '.',
  'O',
  '#',
  'O',
  '#',
  '.',
  '.',
  '.',
  '#',
  '.',
  'O',
  '.',
  'O',
  'O',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  'O',
  'O',
  '.',
  '.',
  '.',
  '.',
  '.',
  'O',
  '.',
  '.',
  '.',
  '.',
  '#'],
 ['#',
  'O',
  '#',
  'O',
  '.',
  '.',
  '.',
  '#',
  '.',
  '.',
  '.',
  '#',
  '.',
  '.',
  '.',
  'O',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  'O',
  'O',
  '.',
  '#',
  '.',
  '.',
  '.',
  'O',
  'O',
  '.',
  '.',
  'O',
  '.',
  '.',
  'O

In [45]:
INSTR

'^>v<v><>^<>v^<vvv<v>v<><>^^v>^^<<^<v^v>v<>v^<^<^v^v^>>v<><>^<^^^^<v^><v<^^>v>vv>v^><<vv>>^><v<^<^v^v>v>>^><>^^vv^<<<>v>v>>vvvv^<^><v>^^><>v>^v^^<>>^<^<v<v^^<v><v><<>>>v^<<>>^^^<^>><v>>v>^>>v^^v^^^>>^v<vv^>>vvv<<>vv<^>>>^^^v<<<^^<v^^<><v>v<v<vvv^>><><>>>^v>^vv>^><^>>vv^^<^<v^<>>v^^vv^v^^v<v^<v^<><>>>>v><<vv<><^vv^vv><^^^vv><v^<^>><<<v^v<>v><>>v^^<^^>><>^vvv<<><<^v<<^>^v^^v><^<^vvv>v<^^vv^<><vvv<><>>>>v<^^v^<vv<v>^v>vv^v>><<v^><<><<>^v>>><^<^<vv>^^^v><^>vv>>v<^<<v>vv<<>>^>v<>^^<^<><^<>vvv><>><<v^^>>>v^>^<<^^>v^>>>vv<^v><>v<<^>>><<><v^><<<<>^>^^v^<>^>v>v><<<>><<>>vv>v><<^><<><vv>vv<v^v^>vv<<v>v^v^v^v<<^v^v><>v>v<^v^>><>><^<>v^^^^v>^^<^v><v<>^<<>^><^^^><>vvv>>>><<vv^v^<v^>vvv^<^^v>><<<v^v^<vv>v^^^^>v<<^<>v<^>^^>^<<v<^^^^>v>>^^<>v>v<<^<><v^^v<v>v>^<^>><<>v>>v>v>^v<<^<<^<vv<>>>vv>^<>v<v^^v<<vv^v>>^v^v>^>^>^>vv><^<<^<><<^v<v<v<v^^vv<<v^>v>v<^>v<vvv^<>^><vv>^v<^>>vvv>v><v>^<>^<^^>><<^><v^>>^v>>><v<^v^<<<<vvvv>^<<<>^>>>v<><><v<v^v>><<^>><v>v^<vv^^<>>^>><<<<^^>v>^^<<v^^>vvv^><v>v^<<>^<<<^<>>^<>

In [46]:
BOX = 'O'
WALL = '#'
ROBOT = '@'
FREE = '.'
LEFT = '<'
RIGHT = '>'
UP = '^'
DOWN = 'v'
DIRS = {LEFT: (0,-1), RIGHT: (0,1), UP: (-1,0), DOWN: (1,0)}

In [47]:
def find_robot(map_data):
    for i, row in enumerate(map_data):
        for j, val in enumerate(row):
            if val == ROBOT:
                return i,j
                
def shift(pos_list, map_data, direction):
    # Set initial robot space to be free
    oi, oj = pos_list[0]
    map_data[oi][oj] = FREE

    # Move the robot
    di, dj = DIRS[direction]
    map_data[oi+di][oj+dj] = ROBOT 

    # If there were boxes, shift all the boxes (effectively adding one box to the end)
    if len(pos_list) > 1:
        fi, fj = pos_list[-1]
        map_data[fi+di][fj+dj] = BOX

    return map_data, (oi+di, oj+dj)
        

def move_robot(pos, map_data, direction):
    curr = pos
    dr, dc = DIRS[direction]
    to_be_shifted = []
    while map_data[curr[0]][curr[1]] != WALL:
        r,c = curr
        if map_data[r][c] == FREE:
            return shift(to_be_shifted, map_data, direction)
        to_be_shifted.append(curr)
        curr = (r+dr, c+dc)
    return map_data, pos

def sum_box_values(map_data):
    total = 0
    for i in range(len(map_data)):
        for j in range(len(map_data[0])):
            if map_data[i][j] == BOX:
                total += 100*i + j
    return total

def simulate():
    map_data = copy.deepcopy(MAP)
    pos = find_robot(map_data)
    # for line in map_data:
    #     print(line)
    for i in INSTR:
        map_data, pos = move_robot(pos, map_data, i)
        # print(i)
        # for line in map_data:
        #     print(line)
        # print()
    return sum_box_values(map_data)

print(simulate())

1429911
